<h3>Usefull stuff</h3>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

def distra(text):
    rem = ['судебного ', 'Судебного ', 'судебный участок ', 'Судебный участок ']
    if 'район' in text:
        if '№ ' in text:
            text = ' '.join(text[text.find('№')+2:text.find('район')+6].split(' ')[1:])
        elif '№' in text:
            text = ' '.join(text[text.find('№')+1:text.find('район')+6].split(' ')[1:])
        else:
            text = text[:text.find('район')+6]
        for i in rem:
            text = text.replace(i, '')
        return text
    elif 'округ' in text:
        if '№ ' in text:
            text = ' '.join(text[text.find('№')+2:text.find('округ')+6].split(' ')[1:])
        elif '№' in text:
            text = ' '.join(text[text.find('№')+1:text.find('округ')+6].split(' ')[1:])
        else:
            text = text[:text.find('округ')+6]
        for i in rem:
            text = text.replace(i, '')
        return text
    else:
        for i in rem:
            text = text.replace(i, '')
        return text

<h3>Dictionary containing of ids of regions we are intrested in</h3>

In [2]:
mother = {'Республика Саха (Якутия)':'14', 'Забайкальский край':'75', 'Республика Бурятия':'04', 'Иркутская область':'38'}

<h3>Collecting data</h3>

In [ ]:
table = []
for k, v in mother.items():
    mother_url = 'https://sudrf.ru/index.php?id=300&act=go_ms_search&searchtype=ms& \
    var=true&ms_type=ms&court_subj={}&ms_city=&ms_street='.format(v)
    headerss = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)  \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    mother_site = requests.get(mother_url, headers = headerss)
    mother_soup = BeautifulSoup(mother_site.content.decode(mother_site.encoding), 'xml')
    for td in mother_soup.find('table', class_='msSearchResultTbl').find_all('TD'):
        try:
            ID = td.find('b').next_sibling.strip()
            url = td.find_all('a')[1]['href'] + '/modules.php?name=terr'
            try:
                site = requests.get(url, headers = headerss)
            except:
                time.sleep(5)
                try:
                    site = requests.get(url, headers = headerss)
                except:
                    print('Чёт всё равно не пошло с {}'.format(url))
                    pass
            soup = BeautifulSoup(site.content.decode(site.encoding), 'xml')
            district = distra(soup.find('meta', {'name':'description'})['content'])
            for i in soup.find_all('div', {'class':'terr-item'}):
                row = []
                text = i.text.replace(k, '')
                for j in district.split(' '):
                    text = text.replace(j, '')
                address = ' '.join([s.strip() for s in text.split('\n') if s != ''])
                row.append(ID)
                row.append(k)
                row.append(district)
                row.append(address)
                row.append(url[:-22])
                table.append(row)
        except Exception as e:
            print(e)
            pass

<h3>Data preprocessing</h3>

In [ ]:
data = table
for i in data:
    try:
        if i[2] != 0:
            if i[3] != 0:
                start = int(i[2])
                end = int(i[3])
                left, right = data[:data.index(i)], data[data.index(i)+1:]
                for j in range(start, end+1):
                    left.append([i[0], ' '.join([i[1].replace('\t', ''), str(j)])
                                 .replace('  ', ' '), 0, 0])
                data = left + right
            else:
                if isinstance(i[2], int) or isinstance(i[2], float):
                    left, right = data[:data.index(i)], data[data.index(i)+1:]
                    left.append([i[0], ' '.join([i[1].replace('\t', ''), i[2].replace(',', '')])
                                 .replace('  ', ' '), 0, 0])
                    data = left + right
                else:
                    lili = i[2].split(',')
                    left, right = data[:data.index(i)], data[data.index(i)+1:]
                    for j in lili:
                        if j != '':
                            if j[0] == ' ':
                                ho = j[1:]
                            else:
                                ho = j
                            left.append([i[0], ' '.join([i[1].replace('\t', ''), ho])
                                         .replace('  ', ' '), 0, 0])
                    data = left + right
    except Exception as e:
        print(i, e)

<h3>Convert to excel</h3>

In [ ]:
pd.DataFrame([[i[0], i[1]] for i in data], 
             columns=['ID', 'Adress']).to_excel(r'C:\Users\Андрей\Desktop\Забайкальский край.xlsx', 
                                                                              index=False, header=True)